# Paso 1.
 Preprocese los datos.
 - Cree la columna 'Age' a partir de la columna 'Year'.
   Asuma que el año actual es 2021.
 - Elimine las columnas 'Year' y 'Car_Name'.

In [1]:

import pandas as pd

data_train = pd.read_csv('../files/input/train_data.csv.zip', index_col = False, compression = "zip")
data_test = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")
print(data_train)

data_train['Age'] = 2021 - data_train['Year']
data_test['Age'] = 2021 - data_test['Year']

data_train.drop(columns=['Year', 'Car_Name'], inplace=True)
data_test.drop(columns=['Year', 'Car_Name'], inplace=True)



                      Car_Name  Year  Selling_Price  Present_Price  \
0                         jazz  2016           7.40          8.500   
1                          i10  2013           4.00          4.600   
2           TVS Apache RTR 180  2011           0.50          0.826   
3                          eon  2016           3.15          4.430   
4    Royal Enfield Thunder 350  2013           1.25          1.500   
..                         ...   ...            ...            ...   
206                        i10  2011           2.55          4.430   
207                    etios g  2015           3.95          6.800   
208           Bajaj Pulsar 150  2006           0.10          0.750   
209                        i20  2014           6.00          7.600   
210                       jazz  2016           6.00          8.400   

     Driven_kms Fuel_Type Selling_type Transmission  Owner  
0         15059    Petrol       Dealer    Automatic      0  
1         30000    Petrol       Deale

In [2]:
import pickle

# Ruta del archivo .pkl
ruta_archivo = "../files/grading/x_train.pkl"

# Abrir y cargar el archivo
with open(ruta_archivo, "rb") as file:
    contenido1 = pickle.load(file)

#contenido1.describe().transpose()

In [3]:
import pickle

# Ruta del archivo .pkl
ruta_archivo = "../files/grading/y_train.pkl"

# Abrir y cargar el archivo
with open(ruta_archivo, "rb") as file:
    contenido2 = pickle.load(file)

#contenido2.describe()

In [4]:
import pickle

# Ruta del archivo .pkl
ruta_archivo = "../files/grading/x_test.pkl"

# Abrir y cargar el archivo
with open(ruta_archivo, "rb") as file:
    contenido3 = pickle.load(file)

#contenido3.describe().transpose()

In [5]:
import pickle

# Ruta del archivo .pkl
ruta_archivo = "../files/grading/y_test.pkl"

# Abrir y cargar el archivo
with open(ruta_archivo, "rb") as file:
    contenido4 = pickle.load(file)

#contenido4.describe()

In [6]:
pd.set_option('display.width', 1000)  
print(data_train.describe().transpose())
print(data_test.describe().transpose())

               count          mean           std      min        25%       50%       75%        max
Selling_Price  211.0      4.692512      4.819333     0.10      1.025      3.75      6.05      23.50
Present_Price  211.0      7.561090      7.382453     0.48      1.365      6.40      9.90      35.96
Driven_kms     211.0  35578.009479  28912.475577  1200.00  15000.000  32000.00  47500.00  213000.00
Owner          211.0      0.047393      0.271907     0.00      0.000      0.00      0.00       3.00
Age            211.0      7.355450      2.794843     3.00      5.000      7.00      9.00      18.00
               count          mean           std     min         25%        50%        75%       max
Selling_Price   90.0      4.588111      5.680985    0.12      0.7850      3.305      5.500      35.0
Present_Price   90.0      7.786444     11.097093    0.32      1.0425      6.385      9.505      92.6
Driven_kms      90.0  40157.211111  55767.215482  500.00  15480.7500  33708.500  50000.000  50000

# Paso 2.
 Divida los datasets en x_train, y_train, x_test, y_test.

In [7]:
x_train = data_train.drop(columns=['Present_Price'])
y_train = data_train["Present_Price"]
x_test = data_test.drop(columns=['Present_Price'])
y_test = data_test["Present_Price"]

# Paso 3.
 Cree un pipeline para el modelo de clasificación. Este pipeline debe
 contener las siguientes capas:
 - Transforma las variables categoricas usando el método
   one-hot-encoding.
 - Escala las variables numéricas al intervalo [0, 1].
 - Selecciona las K mejores entradas.
 - Ajusta un modelo de regresion lineal.

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

categorical_features = ['Fuel_Type', 'Selling_type', 'Transmission']
numerical_features = list(set(x_test.columns) - set(categorical_features))

# Paso 3: Transformador para combinar variables categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', MinMaxScaler(), numerical_features)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),         
    ('feature_selection', SelectKBest(f_regression)), 
    ('regre', LinearRegression()),  
])


# Paso 4.
- Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. Use el error medio absoluto
 para medir el desempeño modelo.

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "feature_selection__k": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    #'regre__fit_intercept': [True, False],
    #'regre__positive' : [True, False],
    #'regre__copy_X' : [True, False]
}

model = GridSearchCV(
    pipeline, param_grid, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1)

model.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission']),
                                                                        ('num',
                                                                         MinMaxScaler(),
                                                                         ['Age',
                                                                          'Driven_kms',
                                                                          'Selling_Price',
                                                                          'Owner'])])),
                                       ('feature_selection',
                                        SelectKBest(score_func=<function f_regression at 0x000001DBB09C3370>)),
                                       ('regre', LinearRegression())]),
             n_jobs=-1,
             param_grid={'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                  11]},
             scoring='neg_mean_absolute_error')

# 4.1
Test de las metricas xD

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

print("Score train", model.score(x_train, y_train), "<", -1.590, model.score(x_train, y_train) < -1.590) 
print("Score test", model.score(x_test, y_test), "<", -2.429, model.score(x_test, y_test) < -2.429)
print()

train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mad = mean_absolute_error(y_train, y_train_pred)

test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mad = mean_absolute_error(y_test, y_test_pred)


print("train_r2", train_r2,">", 0.889, train_r2 > 0.889) 
print("train_mse", train_mse,"<", 5.950, train_mse < 5.950 )
print("train_mad", train_mad,"<", 1.600, train_mad < 1.600 )
print()

print("test_r2",test_r2,">", 0.728, test_r2 > 0.728 )
print("test_mse",test_mse,"<", 32.910, test_mse < 32.910 )
print("test_mad",test_mad,"<", 2.430, test_mad < 2.430 )
print()

print('-------------------------')

Score train -1.621487981615828 < -1.59 True
Score test -2.473619914416375 < -2.429 True

train_r2 0.8916962358587399 > 0.889 True
train_mse 5.8746462805982045 < 5.95 True
train_mad 1.621487981615828 < 1.6 False

test_r2 0.7325716754123306 > 0.728 True
test_mse 32.56667275386626 < 32.91 True
test_mad 2.473619914416375 < 2.43 False

-------------------------


# Paso 5.
 Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
 Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [11]:
import os
import pickle
import gzip

dir_path = '../files/models'

#os.makedirs(os.path.dirname(dir_path), exist_ok=True) # Otra forma de crear/verificar la direccion

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(model, f)
else:
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(model, f)

# Paso 6.
 Calcule las metricas r2, error cuadratico medio, y error absoluto medio
 para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
 files/output/metrics.json. Cada fila del archivo es un diccionario con
 las metricas de un modelo. Este diccionario tiene un campo para indicar
 si es el conjunto de entrenamiento o prueba. Por ejemplo:

 {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
 {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

In [12]:
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
import json

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

train_metrics = {
    "type": "metrics",
    'dataset': 'train',
    'r2': float(r2_score(y_train, y_train_pred)),
    'mse': float(mean_squared_error(y_train, y_train_pred)),
    'mad': float(median_absolute_error(y_train, y_train_pred)),
}

test_metrics = {
    "type": "metrics", 
    'dataset': 'test',
    'r2': float(r2_score(y_test, y_test_pred)),
    'mse': float(mean_squared_error(y_test, y_test_pred)),
    'mad': float(median_absolute_error(y_test, y_test_pred)),
}

output_path = '../files/output/metrics.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True) # Otra forma de crear/verificar la dirección

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(train_metrics, f, ensure_ascii=False) #indent=4
    f.write('\n')
    json.dump(test_metrics, f, ensure_ascii=False) #indent=4
    f.write('\n')


In [13]:
print(model.best_params_)

{'feature_selection__k': 11}
